In [4]:
import pandas as pd

df = pd.read_csv("C:\\Users\\VINOTH\\Downloads\\GUIDE_Test.csv\\GUIDE_Test.csv")
print(df)


C:\Users\VINOTH\AppData\Local\Temp\ipykernel_14612\2694159657.py:3: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:\\Users\\VINOTH\\Downloads\\GUIDE_Test.csv\\GUIDE_Test.csv")


                    Id  OrgId  IncidentId  AlertId                 Timestamp  \
0        1245540519230    657       11767    87199  2024-06-04T22:56:27.000Z   
1        1400159342154      3       91158   632273  2024-06-03T12:58:26.000Z   
2        1279900255923    145       32247   131719  2024-06-08T03:20:49.000Z   
3          60129547292    222       15294   917686  2024-06-12T12:07:31.000Z   
4         515396080539    363        7615     5944  2024-06-06T17:42:05.000Z   
...                ...    ...         ...      ...                       ...   
4147987   541165881145    262        3017   227156  2024-06-04T03:53:15.000Z   
4147988   438086666953   1323       31001   304819  2024-06-04T19:32:19.000Z   
4147989  1297080124487     47        3653     7243  2024-06-15T00:50:22.000Z   
4147990   627065226886     70        4243    19142  2024-06-11T16:18:12.000Z   
4147991   326417516547    227        8384    33529  2024-06-04T18:11:09.000Z   

         DetectorId  AlertTitle        

In [5]:
df.isnull().sum()

Id                          0
OrgId                       0
IncidentId                  0
AlertId                     0
Timestamp                   0
DetectorId                  0
AlertTitle                  0
Category                    0
MitreTechniques       2307104
IncidentGrade               0
ActionGrouped         4146079
ActionGranular        4146079
EntityType                  0
EvidenceRole                0
DeviceId                    0
Sha256                      0
IpAddress                   0
Url                         0
AccountSid                  0
AccountUpn                  0
AccountObjectId             0
AccountName                 0
DeviceName                  0
NetworkMessageId            0
EmailClusterId        4106285
RegistryKey                 0
RegistryValueName           0
RegistryValueData           0
ApplicationId               0
ApplicationName             0
OAuthApplicationId          0
ThreatFamily          4116614
FileName                    0
FolderPath

In [6]:
df.isnull().sum()
df.dropna(thresh=0.9 * len(df), axis=1, inplace=True)  # drop columns with >90% NaNs


In [7]:
df.isnull().sum()

Id                    0
OrgId                 0
IncidentId            0
AlertId               0
Timestamp             0
DetectorId            0
AlertTitle            0
Category              0
IncidentGrade         0
EntityType            0
EvidenceRole          0
DeviceId              0
Sha256                0
IpAddress             0
Url                   0
AccountSid            0
AccountUpn            0
AccountObjectId       0
AccountName           0
DeviceName            0
NetworkMessageId      0
RegistryKey           0
RegistryValueName     0
RegistryValueData     0
ApplicationId         0
ApplicationName       0
OAuthApplicationId    0
FileName              0
FolderPath            0
ResourceIdName        0
OSFamily              0
OSVersion             0
CountryCode           0
State                 0
City                  0
Usage                 0
dtype: int64

In [ ]:
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(categorical_cols)
#df.drop(Usage)

['Timestamp', 'Category', 'IncidentGrade', 'EntityType', 'EvidenceRole', 'Usage']


In [11]:
import pickle
from sklearn.preprocessing import LabelEncoder

with open('label_encoder.pkl', 'rb') as f:
    loaded_encoder = pickle.load(f)
categorical_cols = df.select_dtypes(include='object').columns.tolist()
for col in categorical_cols:
    df[col] = df[col].astype(str)  # Convert to string if needed
    df[col] = loaded_encoder.fit_transform(df[col])


In [15]:
import pickle

# Load model
with open('decision_tree_model.pkl', 'rb') as f:
    dt_loaded = pickle.load(f)

# Load feature names used during training
with open('feature_columns.pkl', 'rb') as f:
    trained_columns = pickle.load(f)

# Prepare input data
X = df.drop(columns=['IncidentGrade'])
y = df['IncidentGrade']

# Ensure same columns and order
for col in trained_columns:
    if col not in X.columns:
        X[col] = 0  # or a sensible default

X = X[trained_columns]  # Reorder columns to match training

# Predict
y_pred = dt_loaded.predict(X)



ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- AccountName
- AccountSid
- AccountUpn
- ApplicationId
- ApplicationName
- ...


In [16]:
# Load trained columns
with open('feature_columns.pkl', 'rb') as f:
    trained_columns = pickle.load(f)

X = df.drop(columns=['IncidentGrade'])
y = df['IncidentGrade']

print("Before adjustment:")
print("Columns in X:", X.columns.tolist())

# Add missing columns if any
for col in trained_columns:
    if col not in X.columns:
        X[col] = 0

# Remove extra columns
X = X[trained_columns]

print("After adjustment:")
print("Columns in X:", X.columns.tolist())

# Predict
y_pred = dt_loaded.predict(X)


Before adjustment:
Columns in X: ['Id', 'OrgId', 'IncidentId', 'AlertId', 'Timestamp', 'DetectorId', 'AlertTitle', 'Category', 'EntityType', 'EvidenceRole', 'DeviceId', 'Sha256', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName', 'NetworkMessageId', 'RegistryKey', 'RegistryValueName', 'RegistryValueData', 'ApplicationId', 'ApplicationName', 'OAuthApplicationId', 'FileName', 'FolderPath', 'ResourceIdName', 'OSFamily', 'OSVersion', 'CountryCode', 'State', 'City', 'Usage']
After adjustment:
Columns in X: ['Id', 'OrgId', 'IncidentId', 'AlertId', 'Timestamp', 'DetectorId', 'AlertTitle', 'Category', 'EntityType', 'EvidenceRole', 'DeviceId', 'Sha256', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName', 'NetworkMessageId', 'RegistryKey', 'RegistryValueName', 'RegistryValueData', 'ApplicationId', 'ApplicationName', 'OAuthApplicationId', 'FileName', 'FolderPath', 'ResourceIdName', 'OSFamily', 'OSVersion', 'C

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- AccountName
- AccountSid
- AccountUpn
- ApplicationId
- ApplicationName
- ...


In [19]:
import pickle
from sklearn.preprocessing import LabelEncoder
with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

# # Step 1: Identify categorical columns
categorical_cols = df.select_dtypes(include='object').columns.tolist()

# # Step 2: Initialize label encoder
# le = LabelEncoder()

# Step 3: Apply label encoding to each categorical column
for col in categorical_cols:
    df[col] = df[col].astype(str)  # Convert to string if needed
    df[col] = le.fit_transform(df[col])


In [18]:
import pickle

# Load model
with open('decision_tree_model.pkl', 'rb') as f:
    dt_loaded = pickle.load(f)

# Load feature names used during training
with open('feature_columns.pkl', 'rb') as f:
    trained_columns = pickle.load(f)


In [17]:
# Split features and target
X = df.drop('IncidentGrade', axis=1)
y = df['IncidentGrade']

# Fit the model
dt_loaded.fit(X, y)

# Predict using the trained model
y_pred =dt_loaded.predict(X)


In [20]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

print("\nClassification Report:")
print(classification_report(y, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y, y_pred))


Accuracy: 1.0

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1752940
           1       1.00      1.00      1.00    902698
           2       1.00      1.00      1.00   1492354

    accuracy                           1.00   4147992
   macro avg       1.00      1.00      1.00   4147992
weighted avg       1.00      1.00      1.00   4147992


Confusion Matrix:
[[1752940       0       0]
 [      0  902698       0]
 [      0       0 1492354]]
